In [53]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [54]:
import numpy as np
import pandas as pd
import ast
import warnings
import joblib
import os
warnings.filterwarnings("ignore")

In [55]:
dataset = pd.read_csv('RegressionTestSet.csv')
dataset.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price
0,8/3/2022,Air India,AI,430,9:55,11h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,21:15,economy,"{'source': 'Chennai', 'destination': 'Mumbai'}","4,357"
1,7/3/2022,Indigo,6E,926,18:30,04h 55m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:25,economy,"{'source': 'Delhi', 'destination': 'Mumbai'}","4,270"
2,24-03-2022,Indigo,6E,6491,13:05,07h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,economy,"{'source': 'Bangalore', 'destination': 'Mumbai'}","3,153"
3,27-03-2022,Air India,AI,473,18:40,22h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,17:05,economy,"{'source': 'Delhi', 'destination': 'Bangalore'}","4,728"
4,7/3/2022,Indigo,6E,684,21:55,01h 15m,non-stop,23:10,economy,"{'source': 'Bangalore', 'destination': 'Hydera...","1,714"


In [56]:
dataset['full_information_timestamp'] = 0
dataset['flight_day'] = 0
dataset['flight_month'] = 0
dataset['week_day_of_flight'] = 0
dataset['departure_time_of_the_day']=0
dataset['arrival_time_of_the_day']=0
dataset['num_of_hours_taken'] =0
dataset['num_of_stops'] = 0
dataset['source'] = 0
dataset['destination'] = 0
dataset['distance_between_2_cities'] = 0
dataset['formatted_price'] =0

In [57]:
def format_date(date):
    dashed_date = date.replace('/','-')
    return dashed_date

In [58]:
dataset['full_information_timestamp'] = dataset.date.apply(lambda date:format_date(date))

In [59]:
def extract_day_month(date):
  splitted_date = date.split('-')
  day = int(splitted_date[0])
  month =int(splitted_date[1])
  if splitted_date[1]== '2' or splitted_date[1]== '02':
      new_date_format = 'February ' + splitted_date[0] + ', '+ splitted_date[2]
  else :
      new_date_format = 'March ' + splitted_date[0] + ', '+ splitted_date[2]
  return day,month,new_date_format

In [60]:
dataset[['flight_day', 'flight_month','full_information_timestamp']] = pd.DataFrame(dataset.full_information_timestamp.apply(lambda date: extract_day_month(date)).tolist(), index=dataset.index)

In [61]:
dataset['full_information_timestamp'] = pd.to_datetime(dataset['full_information_timestamp'])

In [62]:
def extract_weekday(day):
    return day.day_name()

In [63]:
dataset['week_day_of_flight'] = dataset.full_information_timestamp.apply(lambda date:extract_weekday(date))

In [64]:
def remove_low_frequency_airlines():
    high_frequency_airlines =['Vistara','Air India','Indigo']
    dataset ['airline'] = [airline if airline in high_frequency_airlines else 'Other_airline' for airline in dataset['airline']]
   

In [65]:
remove_low_frequency_airlines()

In [66]:
def calculate_time_taken(time_taken):
      time = time_taken.split(' ')
      float_hour = time[0].split('.')
      hours = 0
      if(len(float_hour)>1):
          time[0] = float_hour[0] + 'h'
          time[1]= float_hour[1][:-1] + time[1]
      if time[1][:-1] != '' : 
          if int(time[1][:-1]) >= 40 :
              hours = int(time[0][:-1]) + 1
          else :
              hours = int(time[0][:-1]) 
      else :
          hours = int(time[0][:-1])
      return hours

In [67]:
dataset['num_of_hours_taken'] = dataset.time_taken.apply(lambda time_taken: calculate_time_taken(time_taken))

In [68]:
def categorize_time(time):
    hours = time.split(':')
    hours[0] = int(hours[0])
    if hours[0] <6:
        return 'Early morning'
    elif 6<= hours[0] < 12 :
        return 'Morning'
    elif 12<= hours[0] < 18:
        return 'Afternoon'
    else:
        return 'Evening'

In [69]:
dataset['dep_time'] = dataset.dep_time.apply(lambda time: categorize_time(time))

In [70]:
dataset['arr_time'] = dataset.arr_time.apply(lambda time: categorize_time(time))

In [71]:
def split_num_of_stops(stop):
    if stop[:8] == 'non-stop':
        stop_count = 0
    elif stop[:6] == '1-stop':
        stop_count = 1
    else:
        stop_count = 2
    return stop_count

In [72]:
dataset['num_of_stops']= dataset.stop.apply(lambda stop:split_num_of_stops(stop))

In [73]:
dataset['type'] = [1 if kind_of_trip == 'business' else 0 for kind_of_trip in dataset['type']]

In [74]:
def split_route(route):
    path = ast.literal_eval(route)
    source = path['source']
    destination = path['destination']
    return source,destination

In [75]:
dataset[['source', 'destination']] = pd.DataFrame(dataset.route.apply(lambda route: split_route(route)).tolist(), index=dataset.index)

In [76]:
distances = dict()
distances['Mumbai','Kolkata'] = 2167
distances['Mumbai','Hyderabad'] = 721
distances['Mumbai','Chennai'] = 1344
distances['Mumbai','Bangalore'] = 995
distances['Mumbai','Delhi'] = 1439
distances['Delhi','Bangalore'] = 2169
distances['Delhi','Kolkata'] = 1554
distances['Delhi','Hyderabad'] = 1579
distances['Delhi','Chennai'] = 2202
distances['Bangalore','Kolkata']= 1560
distances['Bangalore','Hyderabad'] = 569
distances['Bangalore','Chennai'] = 348
distances['Kolkata','Hyderabad'] = 1489
distances['Kolkata','Chennai'] = 1663
distances['Hyderabad','Chennai'] = 628

In [77]:
def calculate_distance(source,destination):
    try :
        dist = distances[source,destination]
    except :
        dist = distances[destination,source]
    return dist

In [78]:
dataset['distance_between_2_cities'] = dataset.apply(lambda dataFrame: calculate_distance(dataFrame['source'], dataFrame['destination']), axis=1)
dataset['distance_between_2_cities'] = dataset['distance_between_2_cities']/2202

In [79]:
def fix_price_format(price):
    number = price.split(',')
    cost = number[0] + number[1]
    cost=int(cost)
    return cost

In [80]:
dataset['formatted_price'] = dataset.price.apply(lambda price:fix_price_format(price))

In [81]:
def one_hot_encode_time(time):
    morning =0
    afternoon =0
    evening = 0
    early_morning = 0
    if time =='Early morning':
        early_morning = 1
    elif time == 'Afternoon':
        afternoon =1
    elif time == 'Morning':
        morning = 1
    elif time == 'Evening' :
        evening = 1
    return early_morning,morning,afternoon,evening

In [82]:
dataset[['dep_time_Early morning', 'dep_time_Morning','dep_time_Afternoon','dep_time_Evening']] = pd.DataFrame(dataset.dep_time.apply(lambda time: one_hot_encode_time(time)).tolist(), index=dataset.index)

In [83]:
dataset[['arr_time_Early morning', 'arr_time_Morning','arr_time_Afternoon','arr_time_Evening']] = pd.DataFrame(dataset.arr_time.apply(lambda time: one_hot_encode_time(time)).tolist(), index=dataset.index)

In [84]:
def one_hot_encode_week_day(day):
    sunday = 0
    saturday = 0
    thursday = 0
    tuesday = 0
    if day =='Tuesday' :
        tuesday = 1
    elif day =='Thursday' :
        thursday = 1
    elif day =='Saturday' :
        saturday = 1
    elif day == 'Sunday' :
        sunday = 1
    return saturday,sunday,tuesday,thursday

In [85]:
dataset[['week_day_of_flight_Saturday', 'week_day_of_flight_Sunday','week_day_of_flight_Tuesday','week_day_of_flight_Thursday']] = pd.DataFrame(dataset.week_day_of_flight.apply(lambda day: one_hot_encode_week_day(day)).tolist(), index=dataset.index)

In [86]:
def one_hot_encode_airline(airline):
    vistara = 0
    indigo = 0
    air_india = 0
    other = 0
    if airline == 'Vistara' :
        vistara = 1
    elif airline == 'Indigo' :
        indigo = 1
    elif airline == 'Air India' :
        air_india = 1
    elif airline =='Other_airline' :
        other = 1
    return vistara,air_india,indigo,other

In [87]:
dataset[['airline_Vistara', 'airline_Air India','airline_Indigo','airline_Other_airline']] = pd.DataFrame(dataset.airline.apply(lambda airline: one_hot_encode_airline(airline)).tolist(), index=dataset.index)


In [88]:
def one_hot_encode_city(city):
    delhi = 0
    hyderabad = 0
    mumbai = 0
    bangalore = 0
    kolkata = 0
    chennai =0
    if city == 'Delhi' :
        delhi = 1
    elif city == 'Hyderabad' :
        hyderabad =1
    elif city == 'Mumbai' :
        mumbai = 1
    elif city == 'Bangalore' :
        bangalore = 1
    elif city == 'Kolkata' :
        kolkata = 1
    elif city == 'Chennai' :
        chennai = 1
    return delhi,hyderabad,mumbai,bangalore,kolkata,chennai

In [89]:
dataset[['source_Delhi','source_Hyderabad', 'source_Mumbai','source_Bangalore','source_Kolkata','source_Chennai']] = pd.DataFrame(dataset.source.apply(lambda city: one_hot_encode_city(city)).tolist(), index=dataset.index)

In [90]:
dataset[['destination_Delhi','destination_Hyderabad', 'destination_Mumbai','destination_Bangalore','destination_Kolkata','destination_Chennai']] = pd.DataFrame(dataset.destination.apply(lambda city: one_hot_encode_city(city)).tolist(), index=dataset.index)


In [91]:
dataset

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,...,source_Mumbai,source_Bangalore,source_Kolkata,source_Chennai,destination_Delhi,destination_Hyderabad,destination_Mumbai,destination_Bangalore,destination_Kolkata,destination_Chennai
0,8/3/2022,Air India,AI,430,Morning,11h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,Evening,0,"{'source': 'Chennai', 'destination': 'Mumbai'}",...,0,0,0,1,0,0,1,0,0,0
1,7/3/2022,Indigo,6E,926,Evening,04h 55m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,Evening,0,"{'source': 'Delhi', 'destination': 'Mumbai'}",...,0,0,0,0,0,0,1,0,0,0
2,24-03-2022,Indigo,6E,6491,Afternoon,07h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,Evening,0,"{'source': 'Bangalore', 'destination': 'Mumbai'}",...,0,1,0,0,0,0,1,0,0,0
3,27-03-2022,Air India,AI,473,Evening,22h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,Afternoon,0,"{'source': 'Delhi', 'destination': 'Bangalore'}",...,0,0,0,0,0,0,0,1,0,0
4,7/3/2022,Indigo,6E,684,Evening,01h 15m,non-stop,Evening,0,"{'source': 'Bangalore', 'destination': 'Hydera...",...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,15-02-2022,Air India,AI,9894,Morning,25h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,Morning,0,"{'source': 'Chennai', 'destination': 'Mumbai'}",...,0,0,0,1,0,0,1,0,0,0
19996,19-03-2022,Air India,AI,402,Morning,22h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,Morning,0,"{'source': 'Kolkata', 'destination': 'Chennai'}",...,0,0,1,0,0,0,0,0,0,1
19997,21-02-2022,Vistara,UK,930,Morning,15h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,Evening,0,"{'source': 'Mumbai', 'destination': 'Bangalore'}",...,1,0,0,0,0,0,0,1,0,0
19998,17-03-2022,Air India,AI,537,Afternoon,17h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,Morning,1,"{'source': 'Delhi', 'destination': 'Kolkata'}",...,0,0,0,0,0,0,0,0,1,0


In [92]:
features = ['type', 'flight_month', 'num_of_hours_taken', 'num_of_stops',
       'distance_between_2_cities', 'airline_Air India', 'airline_Indigo',
       'airline_Other_airline', 'airline_Vistara', 'source_Bangalore',
       'source_Chennai', 'source_Delhi', 'source_Hyderabad', 'source_Kolkata',
       'source_Mumbai', 'destination_Bangalore', 'destination_Chennai',
       'destination_Delhi', 'destination_Hyderabad', 'destination_Kolkata',
       'destination_Mumbai', 'week_day_of_flight_Saturday',
       'week_day_of_flight_Sunday', 'week_day_of_flight_Thursday',
       'week_day_of_flight_Tuesday', 'dep_time_Afternoon',
       'dep_time_Early morning', 'dep_time_Evening', 'dep_time_Morning',
       'arr_time_Afternoon', 'arr_time_Early morning', 'arr_time_Evening',
       'arr_time_Morning','formatted_price']

In [93]:
for column in dataset.columns:
    if column not in features:
        dataset.drop(columns=column,axis=1,inplace=True)

In [94]:
dataset = dataset[['type', 'flight_month', 'num_of_hours_taken', 'num_of_stops',
       'distance_between_2_cities', 'airline_Air India', 'airline_Indigo',
       'airline_Other_airline', 'airline_Vistara', 'source_Bangalore',
       'source_Chennai', 'source_Delhi', 'source_Hyderabad', 'source_Kolkata',
       'source_Mumbai', 'destination_Bangalore', 'destination_Chennai',
       'destination_Delhi', 'destination_Hyderabad', 'destination_Kolkata',
       'destination_Mumbai', 'week_day_of_flight_Saturday',
       'week_day_of_flight_Sunday', 'week_day_of_flight_Thursday',
       'week_day_of_flight_Tuesday', 'dep_time_Afternoon',
       'dep_time_Early morning', 'dep_time_Evening', 'dep_time_Morning',
       'arr_time_Afternoon', 'arr_time_Early morning', 'arr_time_Evening',
       'arr_time_Morning','formatted_price']]

In [95]:
label = dataset['formatted_price']
dataset.drop(columns=['formatted_price'],axis=1,inplace=True)
features = dataset.copy()

In [96]:
saved_models_path = 'SavedModels'
for model in os.listdir(saved_models_path):
    model_name = model[:-3]
    print("Using model: ", model_name)
    regressor_path = os.path.join(saved_models_path,model)
    regressor = joblib.load(regressor_path)
    score = regressor.score(features,label)
    print("Testing Score: ", score)
    print("-"*26)

Using model:  BaggingRegressor
Testing Score:  0.9638653151099412
--------------------------
Using model:  DesicionTreeRegressor
Testing Score:  0.9606501828866634
--------------------------
Using model:  EnsembleRegressor
Testing Score:  0.9660718248098504
--------------------------
Using model:  ExtraTreeRegressor
Testing Score:  0.9616495808044131
--------------------------
Using model:  HistGradientBoostingRegressor
Testing Score:  0.959755858506913
--------------------------
Using model:  LightGradientBoostingMachineRegressor
Testing Score:  0.9598600115490242
--------------------------
Using model:  LinearRegressor
Testing Score:  0.9082052307634532
--------------------------
Using model:  PoissonRegressor
Testing Score:  0.9328286897698369
--------------------------
Using model:  RandomForestRegressor
Testing Score:  0.9652625029469881
--------------------------
